In [12]:
import cv2
import os
import time
from waste_predictor import WastePredictor

#### **INFERENCE PARAMETERS**

In [3]:
CLASS_NAMES = ['BEER_CANS', 'OTHER', 'PET_BOTTLES', 'SHAMPOO_BOTTLES', 'YOGHURT_CUPS']

TORCHSCRIPT_MODEL_PATH = 'model_scripted.pt'
PYTORCH_MODEL_PATH = 'runs/dec_1/model_13.pth'

In [4]:
torchscript_predictor = WastePredictor(
    model_path = TORCHSCRIPT_MODEL_PATH,
    num_classes = len(CLASS_NAMES),                    
    is_torchscript = True,
    fp16 = True
)

pytorch_predictor = WastePredictor(
    model_path = PYTORCH_MODEL_PATH,
    num_classes = len(CLASS_NAMES),                    
    is_torchscript = False,
    fp16 = False
)


In [5]:
# Single image prediction
image_path = r'datasets/MY2/TEST/BEER_CANS/metal581.jpg'
cv_image = cv2.imread(image_path)

pred_idx, confidence = torchscript_predictor.predict_single(cv_image, return_confidence=True)
print(f'Prediction: {CLASS_NAMES[pred_idx]}')
print(f'Confidence: {confidence:.2%}')

pred_idx, confidence = pytorch_predictor.predict_single(cv_image, return_confidence=True)
print(f'Prediction: {CLASS_NAMES[pred_idx]}')
print(f'Confidence: {confidence:.2%}')

Prediction: PET_BOTTLES
Confidence: 88.18%
Prediction: PET_BOTTLES
Confidence: 88.10%


#### Compare predictions
1. Model Outputs
2. Model Confidences
3. Model Inference Speed

In [17]:
TEST_IMAGE_DIR = r'datasets/MY2/TEST'

results = []
for class_name in os.listdir(TEST_IMAGE_DIR):
    class_dir = os.path.join(TEST_IMAGE_DIR, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        cv_image = cv2.imread(image_path)

        t1_start = time.time()
        pred1, conf1 = torchscript_predictor.predict_single(cv_image, return_confidence=True)
        t1_end = time.time()

        t2_start = time.time()
        pred2, conf2 = pytorch_predictor.predict_single(cv_image, return_confidence=True)
        t2_end = time.time()

        result = {
            'image_name': image_path,
            'class_name': class_name,
            'torchscript_pred': CLASS_NAMES[pred1],
            'torchscript_conf': round(conf1*100, 2),
            'torchscript_time': t1_end - t1_start,
            'pytorch_pred': CLASS_NAMES[pred2],
            'pytorch_conf': round(conf2*100, 2),
            'pytorch_time': t2_end - t2_start
        }

        results.append(result)

In [18]:
import pandas as pd
df = pd.DataFrame(results)
df

,image_name,class_name,torchscript_pred,torchscript_conf,torchscript_time,pytorch_pred,pytorch_conf,pytorch_time
0,datasets/MY2/TEST\BEER_CANS\metal581.jpg,BEER_CANS,PET_BOTTLES,88.18,0.020976,PET_BOTTLES,88.10,0.033047
1,datasets/MY2/TEST\BEER_CANS\metal612.jpg,BEER_CANS,PET_BOTTLES,88.28,0.020877,PET_BOTTLES,88.26,0.039283
2,datasets/MY2/TEST\BEER_CANS\metal677.jpg,BEER_CANS,SHAMPOO_BOTTLES,54.49,0.019761,SHAMPOO_BOTTLES,54.56,0.032198
3,datasets/MY2/TEST\OTHER\trash10.jpg,OTHER,OTHER,100.00,0.019642,OTHER,100.00,0.032278
4,datasets/MY2/TEST\OTHER\trash8.jpg,OTHER,OTHER,99.51,0.016996,OTHER,99.53,0.006002
5,datasets/MY2/TEST\PET_BOTTLES\plastic523.jpg,PET_BOTTLES,PET_BOTTLES,57.28,0.004999,PET_BOTTLES,57.27,0.004001
6,datasets/MY2/TEST\PET_BOTTLES\plastic85.jpg,PET_BOTTLES,PET_BOTTLES,86.87,0.004448,PET_BOTTLES,86.84,0.005003
7,datasets/MY2/TEST\SHAMPOO_BOTTLES\plastic861.jpg,SHAMPOO_BOTTLES,SHAMPOO_BOTTLES,77.64,0.005999,SHAMPOO_BOTTLES,77.64,0.005003
8,datasets/MY2/TEST\SHAMPOO_BOTTLES\scalpe-plus-...,SHAMPOO_BOTTLES,PET_BOTTLES,87.35,0.007110,PET_BOTTLES,87.34,0.005005
9,datasets/MY2/TEST\YOGHURT_CUPS\plastic362.jpg,YOGHURT_CUPS,SHAMPOO_BOTTLES,47.80,0.005999,SHAMPOO_BOTTLES,47.77,0.006001


**There is no difference in Optimized Model**